In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install pypianoroll

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pypianoroll
import os
from IPython.display import Audio
from random import randrange

device = 'cpu'

In [2]:

class MusicDataset(Dataset):
    def __init__(self, data, seq_length=50):
        self.data = data
        self.seq_length = seq_length

    def __getitem__(self, index):
        train_seq = self.data[:, index * self.seq_length: (index + 1) * self.seq_length, :]
        target_seq = self.data[:, (index + 1) * self.seq_length, :]
        return train_seq, target_seq

    def __len__(self):
        return int(self.data.size(1) / self.seq_length)

class MusicGenerationRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size=32, n_layers=1):
        super(MusicGenerationRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.batch_size = batch_size
        self.gru = nn.GRU(input_size, hidden_size, n_layers)
        self.linear = nn.Linear(hidden_size * n_layers, output_size)

    def forward(self, input, hidden):
        input = input.permute(0, 2, 1, 3)
        input = input.flatten(2, 3)
        input = input.permute(1, 0, 2)
        _, hidden = self.gru(input, hidden)
        h_n = hidden.permute(1, 0, 2)
        h_n = h_n.contiguous().flatten(1, 2)
        output = self.linear(h_n)
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.n_layers, batch_size, self.hidden_size).to(device)


In [4]:
input_path = os.path.join('..', 'data')
preparation_path = os.path.join(input_path, '02_preparation')
model_path = os.path.join(input_path, '03_model', 'rnn')
validation_path = os.path.join(input_path, '04_validation', 'rnn')


In [5]:
torch_tensor = torch.load(os.path.join(preparation_path, 'tensor.pt'))
torch_tensor = torch_tensor.type(torch.float32)

test_dataset = MusicDataset(torch_tensor[:, int(torch_tensor.shape[1] * 0.8):, :], seq_length=64)
test_loader = DataLoader(test_dataset, batch_size=64,
                         shuffle=True)

model = torch.load(os.path.join(model_path, 'model_torch.pt'))
model.to(device)

In [6]:

def evaluate(model, prime_seq, predict_len = 100):
    hidden = model.init_hidden(batch_size = 1)

    predictions = torch.zeros((7, predict_len + prime_seq.size()[1], 72)).to(device)

    predictions[:, 0:prime_seq.size()[1], :] = prime_seq[:, :, :72]
    curr_predict_id = prime_seq.size()[1]
    prime_seq = prime_seq.unsqueeze(0)

    _, hidden = model(prime_seq, hidden)

    input = prime_seq[:, :, -1:, :]

    while curr_predict_id < predictions.size()[1]:
      scores, hidden = model(input, hidden)
      scores[scores < 0.10] = 0.0

      scores = scores.view(7, 72)
      predicted = torch.zeros_like(scores)

      for i in range(7):
        instrument_predicted_n_notes = torch.nonzero(scores[i,:]).shape[0]
        if instrument_predicted_n_notes > 0:
          instrument_predicted_n_notes = randrange(3)
          topk, indices = torch.topk(scores[i, :], instrument_predicted_n_notes)
          predicted[i, indices] = 1

      input = predicted.clone()
      input = input.unsqueeze(0).unsqueeze(2)

      predictions[:, curr_predict_id, :] = predicted
      curr_predict_id += 1

    return predictions

def generate_music():
    dataset = test_loader.dataset
    n_samples = len(dataset)
    test, target = dataset[randrange(n_samples)]
    prime_seq = test
    predictions = evaluate(model, prime_seq, predict_len = 500)
    predictions = (predictions * 127).type(torch.int8)

    piano_track = pypianoroll.StandardTrack(name = 'Piano', program = 0, is_drum = False, pianoroll = predictions[0, :, :])
    piano_track_right = pypianoroll.StandardTrack(name = 'Piano Right', program = 0, is_drum = False, pianoroll = predictions[1, :, :].detach().cpu().numpy())
    piano_track_left = pypianoroll.StandardTrack(name = 'Piano Left', program = 0, is_drum = False, pianoroll = predictions[2, :, :].detach().cpu().numpy())
    violin1_track = pypianoroll.StandardTrack(name = 'Violin1', program = 41, is_drum = False, pianoroll = predictions[3, :, :].detach().cpu().numpy())
    violin2_track = pypianoroll.StandardTrack(name = 'Violin2', program = 41, is_drum = False, pianoroll = predictions[4, :, :].detach().cpu().numpy())
    viola_track = pypianoroll.StandardTrack(name = 'Viola', program = 42, is_drum = False, pianoroll = predictions[5, :, :].detach().cpu().numpy())
    cello_track = pypianoroll.StandardTrack(name = 'Cello', program = 43, is_drum = False, pianoroll = predictions[6, :, :].detach().cpu().numpy())

    generated_multitrack = pypianoroll.Multitrack(name = 'Generated', resolution = 4, tracks = [piano_track,
                                                                                                piano_track_right,
                                                                                                piano_track_left,
                                                                                                violin1_track,
                                                                                                violin2_track,
                                                                                                viola_track,
                                                                                                cello_track,
                                                                                              ])

    generated_multitrack.plot()
    return generated_multitrack

def create_and_write(i):
    generated_multitrack = generate_music()
    pypianoroll.write(os.path.join(validation_path, f'rnn_track_{i}.mid'), generated_multitrack)

In [ ]:
for j in range(0, 3):
    create_and_write(j)
